In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import sunpy.map
from sunpy.coordinates import frames
from sunpy.net import Fido, attrs as a
from astropy import units as u
from astropy.coordinates import SkyCoord
from sunpy import timeseries as ts
from sunpy.coordinates import get_body_heliographic_stonyhurst
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (16,8)

# Aquiring Data and SunPy Data Classes
In this example notebook we will go over the basics of using SunPy to access, download, load and analyse solar data. 

We will first look at how SunPy's FIDO functionality allows us to search and download data using a unified interface, and how we can go about making queries of interest. We will then look at how we can load and analyse timeseries data using the SunPy TimeSeries data class, and similarly use the SunPy Map class to load and analyse image data in a specific coordinate frame.

Lets first search for and download data!

## Fido Unified Downloader

Fido (Federated Internet Data Obtainer) is a unified interface for the search and retrival of solar physics data regardless of the underlying client or webservice through which the data is obtained, e.g. VSO, JSOC, etc. 

In SunPy 1.0 Fido now makes use of an asynchronous download stream through the module `parfive`

### Lets look at how we can search and download some solar physics data

In [ ]:
#lets search for some SDO/AIA data over some time
results = Fido.search(a.Time('2014/05/15 08:00', '2014/05/15 08:10'), a.Instrument('AIA'))

This returns a `UnifiedResponse` object that holds the information about the search 

In [ ]:
results

To find out the number of files returned in the search we use the `.file_num` attribute

In [ ]:
results.file_num

Queries to Fido can be make more specific by using other attributes. Lets narrow the search to only look at one wavelength, with a cadence of 1 minute. This makes use of Astropy units 

In [ ]:
results = Fido.search(a.Time('2014/05/15 08:00', '2014/05/15 08:10'), a.Instrument('AIA'), 
                      a.Wavelength(17.1*u.nm), a.vso.Sample(1*u.minute))

In [ ]:
results

### We can then downloaded the data via Fido's `fetch` function
Once the files of interest have been located from the search query via `Fido.search`, they can be downloaded using `Fido.fetch` :

In [ ]:
aia_files = Fido.fetch(results)

This will download the files to a path defined in your sunpy config file. The function returns a `parfive.Results` object, in this case stored in `aia_files`. We can list `aia_files` to get the file paths of where the data was downloaded

In [ ]:
aia_files[0:10]

If however you want the files to be downloaded to a certain directory, you can specifiy the path for which you would like the data to be downloaded to

In [ ]:
aia_files_specific = Fido.fetch(results, path='~/sunpy/spd/{instrument}')

In [ ]:
aia_files_specific

### More detailed searchs with Fido
Fido searches can include searchs from multiple instruments, wavelengths, times etc. This is achieved through the | opertaior which acts as a logical OR operator.


In [ ]:
results = Fido.search(a.Time('2014/05/15 08:00', '2014/05/15 08:10'), a.Instrument('AIA')|a.Instrument('SWAP'))
   

In [ ]:
# the results can be indexed to access a subset of the search results
results[:,0]

In [ ]:
files = Fido.fetch(results[:,0])

# Representing Data

SunPy provides core data type classes that are designed to provide a consistent interface acress data types (lightcurves and images) as well as data sources from numerous instruments and observations. They handle all of the manipulation necessary to read data in from mission-specific files.  The two main datatypes in SunPy are the TimeSeries and Map classes.

## TimeSeries
The structure of a TimeSeries consists of times and measurements and the underlying structure is that of a `pandas.DataFrame`. SunPy TimeSeries supports time-series data from a wide range of solar-focussed instruments.

The `TimeSeries` class now supersedes the `LightCurve` object which is now deprecated in SunPy 1.0. The main difference is that `TimeSeries` does not download the data, which is now done through `Fido` as described above. 

Lets look at an example of using TimeSeries to deal with GOES/XRS observations. First we query and download the data using `Fido`, and then load the downloaded file into the `TimeSeries` class:

In [ ]:
search_results = Fido.search(a.Time('2013-10-28 00:00','2013-10-28 12:00'), a.Instrument('XRS'))
goes_files = Fido.fetch(search_results[0])

We can now load this downloaded file into `TimeSeries` which will automatically detect the instrument source

In [ ]:
goes_lc = ts.TimeSeries(goes_files)

In [ ]:
goes_lc

In [ ]:
goes_lc.peek()

We can inspect the meta information in the TimeSeries object.

In [ ]:
goes_lc.meta

And inspect the associated units

In [ ]:
goes_lc.units

We can also inspect and manipulate the data within the `TimeSeries` class which is implemented as a `pandas.DataFrame`

In [ ]:
sub_lc = goes_lc.truncate("2013-10-28T03:00", "2013-10-28T06:00")

In [ ]:
sub_lc.peek()